# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import cv2
import os
from scipy.misc import imread, imresize
import datetime
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.regularizers import l2, l1
from keras.layers import Conv3D,MaxPooling3D,SimpleRNN
import random
import cv2

Using TensorFlow backend.


We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
batch_size = 30 #experiment with the batch size(UPDATE THIS-50, 40, 20, 10)

In [4]:
from skimage import transform as tf

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
##Helper functions for Generator
#Get image indexes to be used in the video
def getImageIndexes(type="All"):
    if type=="All":
        return [x for x in range(0, 30)] #return all indexes
    elif type == "Alternate1":
         return [x for x in range(0, 30, 2)] #return alternate frames starting with index 0
    elif type == "Alternate2":
         return [x for x in range(1, 30, 2)] #return alternate frames starting with index 1
    elif type == "Alternate3":
         return [x for x in range(0, 30, 3)] #return alternate frames starting with index 0 and skipping 2 frames in between
    elif type == "Less":
        return [0, 2, 10, 12, 14, 16, 18, 20, 27, 29] #return frame mostly from middle of the video with 2 frames from starting and ending each

In [6]:
##Helper functions for Generator
#Image resize/crop/padding functions
def cropImageTo120(image):
    return image if image.shape[0] == image.shape[1] else image[0:120, 20:140]

def resizeImagetoSmallerSize(image, size=120):
    return image if image.shape[0]==size else cv2.resize(image, (size,size), interpolation = cv2.INTER_AREA) #Shrink image to 120X120

def resizeToSmallerImage(image, size=120):
    #Crop and resize to size
    return resizeImagetoSmallerSize(cropImageTo120(image), size)

def resizeToLargeImage(image, size=360):
    if image.shape[0] == image.shape[1] and image.shape[0] == size:
        return image
    if image.shape[0] != image.shape[1]:
        #pad image to make it of size 160X160
        image = cv2.copyMakeBorder(image,20,20,0,0,cv2.BORDER_CONSTANT,value=BLACK) #Pad image: cv2.copyMakeBorder( src, top, bottom, left, right, borderType)
    if image.shape[0] > size:
        return cv2.resize(image, (size,size), interpolation = cv2.INTER_AREA)
    else:
        return cv2.resize(image, (size,size), interpolation = cv2.INTER_CUBIC)

#Image normalization functions
def normalizeBasic(image):
    return image/255

def normalizeMean(image):
    return (image - (np.min(image)/(np.max(image) - np.min(image))))

def normalizePercentile(image):
    return (image - (np.percentile(image,5)/(np.percentile(image,95) - np.percentile(image,5))))

In [7]:
#Initialize variables used in this model
image_idx=getImageIndexes("Alternate1")
y=80
z=80
input_shape=(15,y,z,3)
normalizationTech="normalizeMean"


In [8]:
#Common code for Generator to load batch images
def load_images_in_batch(source_path, folder_list, batch_size, t, batch, img_idx, train = True):
    #Common code from Generator
    try:
        batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
        batch_data_rot = np.zeros((batch_size,len(img_idx),y,z,3))
        batch_labels_rot = np.zeros((batch_size,5))
        batch_data_affine = np.zeros((batch_size,len(img_idx),y,z,3))
        batch_labels_affine = np.zeros((batch_size,5))
        for folder in range(batch_size): # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes(UPDATE THIS)
                image = resizeToSmallerImage(image, 80)
                normalized_image = normalizeMean(image[:,:,:])

                batch_data[folder,idx,:,:,0] = normalized_image[:,:,0]
                batch_data[folder,idx,:,:,1] = normalized_image[:,:,1]
                batch_data[folder,idx,:,:,2] = normalized_image[:,:,2]
                
                #Augmentation
                #Rotate images by 30 degree
                (h, w) = image.shape[:2]
                (cx, cy) = (w // 2, h // 2)
                M = cv2.getRotationMatrix2D((cx, cy), 30, 1.0)
                batch_data_rot[folder,idx] = cv2.warpAffine(normalized_image, M, (w, h))
                #Affine transformation
                dx, dy = np.random.randint(-1.5, 1.4, 2)
                M = np.float32([[1, 0, dx], [0, 1, dy]])
                batch_data_affine[folder,idx] = (cv2.warpAffine(normalized_image, M, (w,h)))
            
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            batch_labels_rot[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            batch_labels_affine[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
    except Exception as e:
        print(e)
    if train:
        batch_data_final = np.append(batch_data, batch_data_affine, axis = 0)
        batch_data_final = np.append(batch_data_final, batch_data_rot, axis = 0)
        batch_labels_final = np.append(batch_labels, batch_labels_affine, axis = 0) 
        batch_labels_final = np.append(batch_labels_final, batch_labels_rot, axis = 0)
        rand_idxs = random.sample(range(batch_size*3), batch_size)
        return batch_data_final, batch_labels_final
    else:
        return batch_data, batch_labels

In [9]:
def generator(source_path, folder_list, batch_size, train = True):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = getImageIndexes("Alternate1") #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            yield load_images_in_batch(source_path, folder_list, batch_size, t, batch, img_idx, train) #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        last_batch_size = len(folder_list)%batch_size
        yield load_images_in_batch(source_path, folder_list, last_batch_size, t, num_batches, img_idx, train)


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = '/notebooks/storage/Final_data/Collated_training/train'
val_path = '/notebooks/storage/Final_data/Collated_training/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 40 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 40


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [11]:
##CNN+RNN models functions
def getConvRNN(num_filters, shape, num_classes=5):
    model = Sequential()
    
    model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

    model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(BatchNormalization()))
    model.add(Dropout(0.25))

    model.add(TimeDistributed(Flatten()))
    model.add(Dense(128, activation='relu'))# fully connected
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))# fully connected
    model.add(Dropout(0.25))
    model.add(SimpleRNN(num_filters*8,  return_sequences=True))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def getConvBiDirectionalRNN(num_filters, shape, num_classes):
    
    nb_featuremap = [8,16,32,64]
    nb_dense = [128,64,5]
    nb_classes = 5
    model = Sequential()
    model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

    model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(BatchNormalization()))
    model.add(Dropout(0.25))


    model.add(TimeDistributed(Flatten()))
    model.add(Dense(128, activation='relu'))# fully connected
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))# fully connected
    model.add(Dropout(0.25))
    model.add(Bidirectional(SimpleRNN(128,  return_sequences=False)))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def getConvLSTM(num_filters, shape, num_classes, num_cells):
    model = Sequential()
    model.add(Conv2D(num_filters, kernel_size=(3, 3), activation='relu', input_shape=shape))

    model.add(Conv2D(num_filters*2, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(num_filters*3, activation='relu')) # fully connected
    model.add(Dropout(0.5))

    
    model.add(LSTM(num_cells, return_sequences=True)) 
    model.add(TimeDistributed(Dense(num_classes, activation='softmax')))
    return model

def getConvGRU(input_shape):
    nb_featuremap = [16,32,64,128]
    nb_dense = [128,64,5]
    nb_classes = 5
    model = Sequential()
    model.add(TimeDistributed(Conv2D(nb_featuremap[0], (3, 3), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))

    model.add(TimeDistributed(Conv2D(nb_featuremap[1], (3,3),padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(Conv2D(nb_featuremap[2], (3,3),padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(Conv2D(nb_featuremap[3], (2,2),padding='same', activation='relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

    model.add(TimeDistributed(BatchNormalization()))
    model.add(Dropout(0.25))

    model.add(TimeDistributed(Flatten()))
    model.add(Dense(128, activation='relu'))# fully connected
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))# fully connected
    model.add(Dropout(0.25))
    model.add(GRU(64, return_sequences=False)) 
    model.add(Dense(nb_classes, activation='softmax'))
    return model

In [12]:
#Convolution 3d model
def getConv3dModel(input_shape):
    model = Sequential()
    model.add(Conv3D(16, (3,3,3), strides=(1,1,1), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

    model.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

    model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

    model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling3D(pool_size=(1,1,1), strides=(1,1,1)))

    model.add(Flatten())
    model.add(Dropout(0.25))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(5, activation='softmax'))
    return model

In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,GlobalAveragePooling2D
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications.resnet50 import ResNet50
from keras import optimizers 

model = getConv3dModel(input_shape)

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [14]:
#Parameters
print("Image indexes", image_idx)
print("Image shape", input_shape)
print("Normalization technique:", normalizationTech)
print("Batch size:", batch_size)
print("Number of epochs:", num_epochs)

Image indexes [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
Image shape (15, 80, 80, 3)
Normalization technique: normalizeMean
Batch size: 30
Number of epochs: 40


In [15]:
optimiser = 'sgd' #write your optimizer UPDATE THIS (SGD, Adam, Adamax, Adadelta)
print ("Optimiser:", optimiser)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Optimiser: sgd
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 15, 80, 80, 16)    1312      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 80, 80, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 80, 80, 16)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 7, 40, 40, 16)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 7, 40, 40, 32)     13856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 40, 40, 32)     128       
_________________________________________________________________
activation_2 (Activation)    (None, 7, 40, 40, 32)     0     

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size, train= False)

In [17]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.001)# write the REducelronplateau code here (UPDATE THIS)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [19]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  notebooks/storage/Final_data/Collated_training/val ; batch size = 30
Source path =  notebooks/storage/Final_data/Collated_training/train ; batch size = 30
Epoch 1/40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


23/23 [==============================] - 42s 2s/step - loss: 1.6048 - categorical_accuracy: 0.3382 - val_loss: 3.6517 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2019-12-2215_30_57.677617/model-00001-1.59848-0.33836-3.65171-0.22000.h5
Epoch 2/40
23/23 [==============================] - 37s 2s/step - loss: 1.2605 - categorical_accuracy: 0.4363 - val_loss: 1.3466 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2019-12-2215_30_57.677617/model-00002-1.25282-0.44897-1.34664-0.40000.h5
Epoch 3/40
23/23 [==============================] - 37s 2s/step - loss: 1.1334 - categorical_accuracy: 0.5168 - val_loss: 1.4588 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model_init_2019-12-2215_30_57.677617/model-00003-1.11067-0.53695-1.45880-0.44000.h5
Epoch 4/40
23/23 [==============================] - 37s 2s/step - loss: 0.9912 - categorical_accuracy: 0.5893 - val_loss: 1.2415 - val_categorical_accuracy: 0.4800

Epoch 00004:


Epoch 00030: saving model to model_init_2019-12-2215_30_57.677617/model-00030-0.39305-0.86023-0.49555-0.83000.h5
Epoch 31/40
23/23 [==============================] - 38s 2s/step - loss: 0.3965 - categorical_accuracy: 0.8564 - val_loss: 0.6437 - val_categorical_accuracy: 0.7700

Epoch 00031: saving model to model_init_2019-12-2215_30_57.677617/model-00031-0.38441-0.86375-0.64373-0.77000.h5
Epoch 32/40
23/23 [==============================] - 38s 2s/step - loss: 0.3979 - categorical_accuracy: 0.8450 - val_loss: 0.5635 - val_categorical_accuracy: 0.8000

Epoch 00032: saving model to model_init_2019-12-2215_30_57.677617/model-00032-0.39426-0.84766-0.56352-0.80000.h5
Epoch 33/40
23/23 [==============================] - 38s 2s/step - loss: 0.3930 - categorical_accuracy: 0.8571 - val_loss: 0.6370 - val_categorical_accuracy: 0.7200

Epoch 00033: saving model to model_init_2019-12-2215_30_57.677617/model-00033-0.38801-0.86023-0.63696-0.72000.h5
Epoch 34/40
23/23 [==============================